In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

# Load your dataset
df = pd.read_csv('it_problems.csv')

# Tokenize the problem descriptions
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['problem_description'])
sequences = tokenizer.texts_to_sequences(df['problem_description'])

# Pad the sequences to ensure uniform input length
x_data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=200)

# Convert the solutions to categorical data
y_data = pd.get_dummies(df['solution']).values

# Build the model
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=64, input_length=200),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_data.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(x_data, y_data, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_data, y_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Make predictions
new_problems = ["Example problem description"]
new_sequences = tokenizer.texts_to_sequences(new_problems)
new_padded = keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=200)

predictions = model.predict(new_padded)
predicted_solution = np.argmax(predictions, axis=1)

print(predicted_solution)

# Save the model
model.save('it_problem_solver_model.h5')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 64)           640000    
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 677,509
Trainable params: 677,509
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 1.6144 - accuracy: 0.0000e+00 - val_loss: 1.6185 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [===================

In [12]:
import pandas as pd

# Define the data for the dataset
data = {
    'problem_description': [
        "How to fix a '404 Not Found' error?",
        "How to recover a lost password on Windows?",
        "How to install Python on Linux?",
        "How to set up a virtual environment in Python?",
        "How to resolve a 'Connection refused' error in a web server?"
    ],
    'correct_answers': [
        "Check if the URL is correct or if the page has been moved.|Ensure the server is running and the path is correct.|Clear the browser cache and refresh.",
        "Use the password reset feature on the login screen.|Follow the steps provided in the recovery email.|Use the Windows security questions if set up.",
        "Use the package manager to install Python, e.g., `sudo apt-get install python3`.|Download and install from the official Python website.|Use `conda install python` if using Anaconda.",
        "Use `venv` or `virtualenv` to create an isolated environment.|Run `python -m venv myenv` to set up the environment.|Activate the environment with `source myenv/bin/activate`.",
        "Check if the web server is running and if the firewall settings are correct.|Verify that the server is listening on the correct port.|Check server logs for errors."
    ]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('it_problems_with_multiple_answers.csv', index=False)

print("CSV file 'it_problems_with_multiple_answers.csv' has been created.")


CSV file 'it_problems_with_multiple_answers.csv' has been created.


In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Load the dataset
df = pd.read_csv('it_problems_with_multiple_answers.csv')

In [6]:
def get_embeddings(texts):
    # Generate embeddings for a list of texts
    return embed(texts)

def find_most_similar_answer(answer, correct_answers):
    # Embed the provided answer and the correct answers
    answers_to_compare = correct_answers + [answer]
    embeddings = get_embeddings(answers_to_compare)
    
    # Compute cosine similarity between the provided answer and each correct answer
    similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])[0]
    
    # Find the index of the most similar correct answer
    most_similar_index = np.argmax(similarities)
    
    return most_similar_index, similarities[most_similar_index]

def ask_question(question, correct_answers):
    print("\nQuestion: ", question)
    answer1 = input("Player 1, your answer: ")
    answer2 = input("Player 2, your answer: ")
    
    # Find the most similar correct answer for each player's answer
    index1, similarity1 = find_most_similar_answer(answer1, correct_answers)
    index2, similarity2 = find_most_similar_answer(answer2, correct_answers)
    
    return similarity1, similarity2, index1, index2

def evaluate_answers(similarity1, similarity2, correct_answers, index1, index2):
    # Compare the similarities and determine the winner
    print(f"score1 : {similarity1} \n score2: {similarity2}")
    if similarity1 > similarity2:
        return "Player 1 wins with answer closest to: " + correct_answers[index1]
    elif similarity2 > similarity1:
        return "Player 2 wins with answer closest to: " + correct_answers[index2]
    else:
        return "It's a tie!"

def display_menu():
    print("Welcome to the IT Problem-Solving Game!")
    print("1. Start Game")
    print("2. Exit")
    choice = input("Enter your choice: ")
    return choice

def main():
    while True:
        choice = display_menu()
        if choice == '1':
            # Select a random question from the dataset
            question_row = df.sample(n=1).iloc[0]
            question = question_row['problem_description']
            correct_answers = question_row['correct_answers'].split('|')
            
            # Ask the question and get answers
            similarity1, similarity2, index1, index2 = ask_question(question, correct_answers)
            
            # Evaluate answers and display results
            result = evaluate_answers(similarity1, similarity2, correct_answers, index1, index2)
            print(result)
        elif choice == '2':
            print("Thanks for playing!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit

Question:  How to set up a virtual environment in Python?
score1 : 0.24744567275047302 
 score2: 0.20294995605945587
Player 1 wins with answer closest to: Run `python -m venv myenv` to set up the environment.
Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit
Thanks for playing!
